# Import utilities

In [16]:
from community_utils_3layers import *

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Preprocessing

Read the graph

In [ ]:
str_graph = nx.Graph()
att_graph = nx.Graph()

In [ ]:
u1, u2 = [0, 0, 0, 1, 1, 2, 2, 3, 3], [5, 5, 5, 5, 5]
v1, v2 = [1, 2, 3, 2, 4, 3, 4, 4, 5], [0, 1, 2, 3, 4]
w1 = [1/9, 1/9, 1/9, 1/9, 1/9, 1/9, 1/9, 1/9, 1/9]
w2 = [1/5, 1/5, 1/5, 1/5, 1/5]

In [ ]:
u1, u2 = [0, 1, 0, 3], [2, 2, 1, 1]
v1, v2 = [1, 2, 2, 4], [3, 4, 3, 4]
w1 = [1/4, 1/4, 1/4, 1/4]
w2 = [1/4, 1/4, 1/4, 1/4]

att_graph.add_node(0)

In [ ]:
for u, v, w in zip(u1, v1, w1):
    str_graph.add_edge(u, v, weight=w)
for u, v, w in zip(u2, v2, w2):
    att_graph.add_edge(u, v, weight=w)

In [3]:
G = nx.Graph()
G.add_edge(0, 1, weight=0.6)
G.add_edge(0, 2, weight=0.2)
G.add_edge(2, 3, weight=0.1)
G.add_edge(2, 4, weight=0.7)
G.add_edge(2, 5, weight=0.9)
G.add_edge(0, 3, weight=0.3)
str_graph = G.copy()

G = nx.Graph()
G.add_edge(0, 1, weight=0.1)
G.add_edge(2, 3, weight=0.3)
G.add_edge(5, 4, weight=0.2)
G.add_edge(2, 5, weight=0.9)
G.add_edge(0, 3, weight=0.13)
att_graph = G.copy()

In [4]:
str_graph = normalize_to_fixed_weight_nx(str_graph, 1, show=True)
att_graph = normalize_to_fixed_weight_nx(att_graph, 1, show=True)

The whole weight has been equalt to 2.8000000000000003 . Now it is equal to 0.9999999999999999
The whole weight has been equalt to 1.6300000000000001 . Now it is equal to 0.9999999999999999


# Brute force experiments

In [9]:
graph_layers = {'str': str_graph, 'att': att_graph}
graph_names = graph_layers.keys()
node_list = graph_layers[take_first(graph_layers)].nodes()

alphas_grid = [(val, 1-val) for val in np.linspace(0, 1, 21)]
alpha_order = {key: val for val, key in enumerate(graph_names)}

brute_partitions = {name: brute_force(name, graph_layers, alphas_grid, alpha_order, node_list) for name in ['sf', 'ef', 'lf']}

In [11]:
modularity_set_sf = {name: define_modularity_set('sf', brute_partitions[name], graph_layers, alpha_order, node_list) 
                     for name in ['sf', 'ef', 'lf']}
plot_dots_2d(modularity_set_sf, r'$Q_{SF}$', 'data/plots/toy1/toy1_sf.pdf')

In [12]:
modularity_set_ef = {name: define_modularity_set('ef', brute_partitions[name], graph_layers, alpha_order, node_list) 
                     for name in ['sf', 'ef', 'lf']}
plot_dots_2d(modularity_set_ef, '$Q_{EF}$', 'data/plots/toy1/toy1_ef.pdf')

In [13]:
modularity_set_lf = {name: define_modularity_set('lf', brute_partitions[name], graph_layers, alpha_order, node_list) 
                     for name in ['sf', 'ef', 'lf']}
plot_dots_2d(modularity_set_lf, '$Q_{LF}$', 'data/plots/toy1/toy1_lf.pdf')

## Delta

In [14]:
delta = {name: delta_experiment(brute_partitions[name], graph_layers['str'], graph_layers['att']) 
         for name in brute_partitions.keys()}
plot_dots_2d(delta, '$\Delta$', 'data/plots/toy1/toy1_delta.pdf')

## Delta'

In [ ]:
best_partit_phase1 = {layer: define_the_best_partitions_ef(graph_layers[layer]) for layer in ['str', 'att']}
partition_combinations_phase1 = list(product(*best_partit_phase1.values()))  # tuples of partitions

In [ ]:
result_new_delta = {'sf': ([], []), 'ef': ([], []), 'lf': ([], [])}
for partition_phase1_pair in partition_combinations_phase1:
    partition_phase1_pair_dict = {'str': partition_phase1_pair[0], 'att': partition_phase1_pair[1]}
    new_graph = {key: late_fusion_transient_absorption(graph_layers[key], partition_phase1_pair_dict[key]) 
                 for key in graph_layers.keys()}
    for method in ['sf', 'ef', 'lf']:
        delta_on_new_graph = delta_experiment(brute_partitions[method], new_graph['str'], new_graph['att'])
        result_new_delta[method][0].extend(delta_on_new_graph[0])
        result_new_delta[method][1].extend(delta_on_new_graph[1])

plot_dots(result_new_delta, '$\Delta`$', 'data/plots/toy1/toy1_delta_new.pdf')

## Theta

In [ ]:
partition_combinations_phase1 = list(product(*best_partit_phase1.values()))  # tuples of partitions
result_theta = {'sf': ([], []), 'ef': ([], []), 'lf': ([], [])}
for par_phase1_pair in tqdm_notebook(partition_combinations_phase1, leave=False):
    for method in ['sf', 'ef', 'lf']:
        theta = theta_experiment(brute_partitions[method], {'str': par_phase1_pair[0], 'att': par_phase1_pair[1]}, graph_layers)
        result_theta[method][0].extend(theta[0])
        result_theta[method][1].extend(theta[1])
        
plot_dots(result_theta, '$\Theta$', 'data/plots/toy1/toy1_theta.pdf')

https://deftpdf.com/ru/crop-pdf

1 0.8 0.35 0